In [53]:
import numpy as np
import pandas as pd

In [54]:
regular_travel = pd.read_csv('C:/Users/Ajay/Desktop/data_science/data-science-lab-team-04/experiments/data/db_regio/regular_travel.csv', parse_dates=['date'])
regular_travel_test = pd.read_csv('C:/Users/Ajay/Desktop/data_science/data-science-lab-team-04/experiments/data/db_regio/regular_travel_test.csv', parse_dates=['date'])


In [55]:
regular_travel.shape

(420000, 4)

In [56]:
regular_travel_test.shape

(109200, 4)

## Label Incoding EZone feature

In [57]:
from sklearn.preprocessing import LabelEncoder

In [58]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
labelencoder = labelencoder.fit(regular_travel['EZone'])
regular_travel['EZone'] = labelencoder.transform(regular_travel['EZone'])

In [59]:
regular_travel

,date,EZone,hour,Passengers
0,2019-01-01,0,0,0.0
1,2019-01-01,0,1,0.0
2,2019-01-01,0,2,0.0
3,2019-01-01,0,3,0.0
4,2019-01-01,0,4,0.0
...,...,...,...,...
419995,2019-12-16,49,19,NaN
419996,2019-12-16,49,20,NaN
419997,2019-12-16,49,21,NaN
419998,2019-12-16,49,22,NaN


## First Implimentation of ARIMA (all data)

In [60]:
!pip install pmdarima

In [61]:
model =auto_arima(y=y,X=X)

NameError: name 'y' is not defined

In [62]:
from typing import Union


from pandas.core.dtypes.common import is_datetime_or_timedelta_dtype
from sklearn.model_selection._split import _BaseKFold


class GroupedTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator

    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals, in train/test sets.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate. Test splits are treated preferentially,
    i.e. if there are not enough samples the test split is filled first.
    This splitter works on a date range, which is not necessarily aligned to the
    available data if it has missing days.

    ----------

    train_window : int, default=21
        Maximum size for a single training set.

    test_window : int, default=7
        Used to set the size of the test set.

    train_gap : int, default=0
        Gap (in days) before the training set.

    test_gap : int, default=0
        Gap (in days) between the training and test set.
    """

    def __init__(self, train_window: int = 21, test_window=7, train_gap: int = 0, test_gap: int = 0):
        self.train_window = train_window
        self.test_window = test_window
        self.train_gap = train_gap
        self.test_gap = test_gap
        self.n_folds_ = None

    def split(self, X: pd.DataFrame, y, dates:Union[pd.Series, np.ndarray], *_):
        """Generate indices to split data into training and test set according to provided dates.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where `n_samples` is the number of samples
            and `n_features` is the number of features.

        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.

        dates : array-like of shape (n_samples,)
            Dates of the samples, can be passed to sklearn via the `groups` parameter.

        Yields
        ------
        train : ndarray
            The training set indices for that split.

        test : ndarray
            The testing set indices for that split.
        """
        if not is_datetime_or_timedelta_dtype(dates):
            dates = pd.to_datetime(dates)

        start_date, end_date = dates.min(), dates.max()
        date_range = pd.date_range(start_date, end_date)
        n_dates = len(date_range)

        indices = np.arange(n_dates)
        train_starts = range(0, n_dates, self.train_window + self.test_window + self.train_gap + self.test_gap)

        self.n_folds_ = len(train_starts)

        for train_start in train_starts:
            avail_days = min(n_dates - train_start,
                             self.train_window + self.test_window + self.train_gap + self.test_gap)
            test_start = max(train_start, train_start + avail_days - self.test_window - self.train_gap)
            train_dates = date_range[indices[train_start: test_start - self.test_gap]]
            test_dates = date_range[test_start: train_start + avail_days - self.train_gap]
            train_indices = np.where(np.isin(dates, train_dates))[0]
            test_indices = np.where(np.isin(dates, test_dates))[0]
            if len(test_dates) < self.test_window:
                continue
            yield list(train_indices), list(test_indices)

In [63]:
tscv = GroupedTimeSeriesSplit(train_window=21, test_window=7, train_gap = 0)

In [64]:
regular_travel

,date,EZone,hour,Passengers
0,2019-01-01,0,0,0.0
1,2019-01-01,0,1,0.0
2,2019-01-01,0,2,0.0
3,2019-01-01,0,3,0.0
4,2019-01-01,0,4,0.0
...,...,...,...,...
419995,2019-12-16,49,19,NaN
419996,2019-12-16,49,20,NaN
419997,2019-12-16,49,21,NaN
419998,2019-12-16,49,22,NaN


## ARIMA on a single Bus stop 

In [65]:
regular_travel = pd.read_csv('C:/Users/Ajay/Desktop/data_science/data-science-lab-team-04/experiments/data/db_regio/regular_travel.csv')


In [66]:
regular_travel['EZone'] = labelencoder.transform(regular_travel['EZone'])
reg_trav_bus_stop0=regular_travel[regular_travel['EZone']==0]

In [16]:
reg_trav_bus_stop0

,date,EZone,hour,Passengers
0,2019-01-01,0,0,0.0
1,2019-01-01,0,1,0.0
2,2019-01-01,0,2,0.0
3,2019-01-01,0,3,0.0
4,2019-01-01,0,4,0.0
...,...,...,...,...
418819,2019-12-16,0,19,NaN
418820,2019-12-16,0,20,NaN
418821,2019-12-16,0,21,NaN
418822,2019-12-16,0,22,NaN


In [67]:
reg_trav_bus_stop0.reset_index(inplace=True,drop=True)
reg_trav_bus_stop0

,date,EZone,hour,Passengers
0,2019-01-01,0,0,0.0
1,2019-01-01,0,1,0.0
2,2019-01-01,0,2,0.0
3,2019-01-01,0,3,0.0
4,2019-01-01,0,4,0.0
...,...,...,...,...
8395,2019-12-16,0,19,NaN
8396,2019-12-16,0,20,NaN
8397,2019-12-16,0,21,NaN
8398,2019-12-16,0,22,NaN


In [68]:
tscv = GroupedTimeSeriesSplit(train_window=21, test_window=7, train_gap = 0)

Verify train and test indices

In [69]:
for train_ind,test_ind in tscv.split(reg_trav_bus_stop0, y=reg_trav_bus_stop0.Passengers, dates = reg_trav_bus_stop0.date):
  print(reg_trav_bus_stop0.iloc[train_ind])

           date  EZone  hour  Passengers

0    2019-01-01      0     0         0.0

1    2019-01-01      0     1         0.0

2    2019-01-01      0     2         0.0

3    2019-01-01      0     3         0.0

4    2019-01-01      0     4         0.0

..          ...    ...   ...         ...

499  2019-01-21      0    19         0.0

500  2019-01-21      0    20         0.0

501  2019-01-21      0    21         0.0

502  2019-01-21      0    22         0.0

503  2019-01-21      0    23         0.0



[504 rows x 4 columns]

            date  EZone  hour  Passengers

672   2019-01-29      0     0         0.0

673   2019-01-29      0     1         0.0

674   2019-01-29      0     2         0.0

675   2019-01-29      0     3         0.0

676   2019-01-29      0     4         0.0

...          ...    ...   ...         ...

1171  2019-02-18      0    19         0.0

1172  2019-02-18      0    20         0.0

1173  2019-02-18      0    21         0.0

1174  2019-02-18      0    22         0.

In [70]:
#create a copy of the original dataset
df_train=reg_trav_bus_stop0.copy()

In [71]:
df_train.isna().sum()

date             0
EZone            0
hour             0
Passengers    2184
dtype: int64

In [72]:
#concat hour and date columns
df_train['date'] = pd.to_datetime(df_train['date'] + df_train['hour'].apply(lambda hour: str(hour).zfill(2)), format='%Y-%m-%d%H')
df_train 

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
8395,2019-12-16 19:00:00,0,19,NaN
8396,2019-12-16 20:00:00,0,20,NaN
8397,2019-12-16 21:00:00,0,21,NaN
8398,2019-12-16 22:00:00,0,22,NaN


In [73]:
df_train.dropna(subset=['Passengers'],inplace=True)
df_train

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
8227,2019-12-09 19:00:00,0,19,0.0
8228,2019-12-09 20:00:00,0,20,0.0
8229,2019-12-09 21:00:00,0,21,0.0
8230,2019-12-09 22:00:00,0,22,0.0


In [74]:
#Same work with test data
regular_travel_test = pd.read_csv('C:/Users/Ajay/Desktop/data_science/data-science-lab-team-04/experiments/data/db_regio/regular_travel_test.csv')
regular_travel_test['EZone'] = labelencoder.transform(regular_travel_test['EZone'])
df_test=regular_travel_test[regular_travel_test['EZone']==0]
df_test

,date,EZone,hour,Passengers
0,2019-01-22,0,0,0
1,2019-01-22,0,1,0
2,2019-01-22,0,2,0
3,2019-01-22,0,3,0
4,2019-01-22,0,4,0
...,...,...,...,...
108019,2019-12-16,0,19,0
108020,2019-12-16,0,20,0
108021,2019-12-16,0,21,0
108022,2019-12-16,0,22,0


In [75]:
df_test['date'] = pd.to_datetime(df_test['date'] + df_test['hour'].apply(lambda hour: str(hour).zfill(2)), format='%Y-%m-%d%H')
df_test

C:\Users\Ajay\AppData\Local\Temp\ipykernel_4388\2767472790.py:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_test['date'] = pd.to_datetime(df_test['date'] + df_test['hour'].apply(lambda hour: str(hour).zfill(2)), format='%Y-%m-%d%H')


,date,EZone,hour,Passengers
0,2019-01-22 00:00:00,0,0,0
1,2019-01-22 01:00:00,0,1,0
2,2019-01-22 02:00:00,0,2,0
3,2019-01-22 03:00:00,0,3,0
4,2019-01-22 04:00:00,0,4,0
...,...,...,...,...
108019,2019-12-16 19:00:00,0,19,0
108020,2019-12-16 20:00:00,0,20,0
108021,2019-12-16 21:00:00,0,21,0
108022,2019-12-16 22:00:00,0,22,0


**Concat of test and train data**

In [76]:
df_busstop0=pd.concat([df_train,df_test])

In [77]:
df_busstop0

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
108019,2019-12-16 19:00:00,0,19,0.0
108020,2019-12-16 20:00:00,0,20,0.0
108021,2019-12-16 21:00:00,0,21,0.0
108022,2019-12-16 22:00:00,0,22,0.0


In [78]:
df_busstop0.sort_values(by='date',ascending=True,inplace=True)
df_busstop0

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
108019,2019-12-16 19:00:00,0,19,0.0
108020,2019-12-16 20:00:00,0,20,0.0
108021,2019-12-16 21:00:00,0,21,0.0
108022,2019-12-16 22:00:00,0,22,0.0


In [79]:
df_busstop0.reset_index(inplace=True,drop=True)
df_busstop0

,date,EZone,hour,Passengers
0,2019-01-01 00:00:00,0,0,0.0
1,2019-01-01 01:00:00,0,1,0.0
2,2019-01-01 02:00:00,0,2,0.0
3,2019-01-01 03:00:00,0,3,0.0
4,2019-01-01 04:00:00,0,4,0.0
...,...,...,...,...
8395,2019-12-16 19:00:00,0,19,0.0
8396,2019-12-16 20:00:00,0,20,0.0
8397,2019-12-16 21:00:00,0,21,0.0
8398,2019-12-16 22:00:00,0,22,0.0


 # Upsampling the minority class

In [33]:
second_month_train=df_busstop0[672:1176]
majority_class=second_month_train[second_month_train['Passengers']<2]
len(majority_class)

484

In [34]:
minority_class = second_month_train[second_month_train['Passengers'] >1]
minority_class_upsampled = minority_class.sample(replace=True, n=len(majority_class))

In [35]:
minority_class_upsampled

,date,EZone,hour,Passengers
881,2019-02-06 17:00:00,0,17,2.0
877,2019-02-06 13:00:00,0,13,2.0
754,2019-02-01 10:00:00,0,10,3.0
683,2019-01-29 11:00:00,0,11,3.0
851,2019-02-05 11:00:00,0,11,2.0
...,...,...,...,...
881,2019-02-06 17:00:00,0,17,2.0
1167,2019-02-18 15:00:00,0,15,2.0
680,2019-01-29 08:00:00,0,8,4.0
759,2019-02-01 15:00:00,0,15,2.0


In [36]:
data_upsampled = pd.concat([second_month_train, minority_class_upsampled])
data_upsampled

,date,EZone,hour,Passengers
672,2019-01-29 00:00:00,0,0,0.0
673,2019-01-29 01:00:00,0,1,0.0
674,2019-01-29 02:00:00,0,2,0.0
675,2019-01-29 03:00:00,0,3,0.0
676,2019-01-29 04:00:00,0,4,0.0
...,...,...,...,...
881,2019-02-06 17:00:00,0,17,2.0
1167,2019-02-18 15:00:00,0,15,2.0
680,2019-01-29 08:00:00,0,8,4.0
759,2019-02-01 15:00:00,0,15,2.0


In [37]:
data_upsampled.sort_values(by='date',inplace=True)

# Creating some lags informations mean and std

In [38]:
data_upsampled['Passengers_lag_mean_24']=data_upsampled['Passengers'].rolling(window=24).mean()
data_upsampled['Passengers_lag_mean_48']=data_upsampled['Passengers'].rolling(window=48).mean()
data_upsampled

,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_mean_48
672,2019-01-29 00:00:00,0,0,0.0,NaN,NaN
673,2019-01-29 01:00:00,0,1,0.0,NaN,NaN
674,2019-01-29 02:00:00,0,2,0.0,NaN,NaN
675,2019-01-29 03:00:00,0,3,0.0,NaN,NaN
676,2019-01-29 04:00:00,0,4,0.0,NaN,NaN
...,...,...,...,...,...,...
1171,2019-02-18 19:00:00,0,19,0.0,1.791667,1.895833
1172,2019-02-18 20:00:00,0,20,0.0,1.708333,1.854167
1173,2019-02-18 21:00:00,0,21,0.0,1.625000,1.812500
1174,2019-02-18 22:00:00,0,22,0.0,1.541667,1.770833


In [39]:
data_upsampled['Passengers_lag_std_24']=data_upsampled['Passengers'].rolling(window=24).std()
data_upsampled['Passengers_lag_std_48']=data_upsampled['Passengers'].rolling(window=48).std()
data_upsampled

,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_mean_48,Passengers_lag_std_24,Passengers_lag_std_48
672,2019-01-29 00:00:00,0,0,0.0,NaN,NaN,NaN,NaN
673,2019-01-29 01:00:00,0,1,0.0,NaN,NaN,NaN,NaN
674,2019-01-29 02:00:00,0,2,0.0,NaN,NaN,NaN,NaN
675,2019-01-29 03:00:00,0,3,0.0,NaN,NaN,NaN,NaN
676,2019-01-29 04:00:00,0,4,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1171,2019-02-18 19:00:00,0,19,0.0,1.791667,1.895833,0.588230,0.424744
1172,2019-02-18 20:00:00,0,20,0.0,1.708333,1.854167,0.690253,0.504852
1173,2019-02-18 21:00:00,0,21,0.0,1.625000,1.812500,0.769670,0.570787
1174,2019-02-18 22:00:00,0,22,0.0,1.541667,1.770833,0.832971,0.627036


In [40]:
data_upsampled.isna().sum()

date                       0
EZone                      0
hour                       0
Passengers                 0
Passengers_lag_mean_24    23
Passengers_lag_mean_48    47
Passengers_lag_std_24     23
Passengers_lag_std_48     47
dtype: int64

In [41]:
data_upsampled.fillna(value=0,inplace=True)

In [42]:
data_upsampled.isna().sum()

date                      0
EZone                     0
hour                      0
Passengers                0
Passengers_lag_mean_24    0
Passengers_lag_mean_48    0
Passengers_lag_std_24     0
Passengers_lag_std_48     0
dtype: int64

In [43]:
data_upsampled.shape

(988, 8)

In [44]:
data_second_test=df_busstop0[1176:1344]

In [45]:
data_second_test

,date,EZone,hour,Passengers
1176,2019-02-19 00:00:00,0,0,0.0
1177,2019-02-19 01:00:00,0,1,0.0
1178,2019-02-19 02:00:00,0,2,0.0
1179,2019-02-19 03:00:00,0,3,0.0
1180,2019-02-19 04:00:00,0,4,0.0
...,...,...,...,...
1339,2019-02-25 19:00:00,0,19,0.0
1340,2019-02-25 20:00:00,0,20,0.0
1341,2019-02-25 21:00:00,0,21,0.0
1342,2019-02-25 22:00:00,0,22,0.0


In [46]:
data_second_test['Passengers_lag_mean_24']=data_second_test['Passengers'].rolling(window=24).mean()
data_second_test['Passengers_lag_mean_48']=data_second_test['Passengers'].rolling(window=48).mean()
data_second_test

C:\Users\Ajay\AppData\Local\Temp\ipykernel_4388\319067494.py:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_second_test['Passengers_lag_mean_24']=data_second_test['Passengers'].rolling(window=24).mean()

C:\Users\Ajay\AppData\Local\Temp\ipykernel_4388\319067494.py:2: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_second_test['Passengers_lag_mean_48']=data_second_test['Passengers'].rolling(window=48).mean()


,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_mean_48
1176,2019-02-19 00:00:00,0,0,0.0,NaN,NaN
1177,2019-02-19 01:00:00,0,1,0.0,NaN,NaN
1178,2019-02-19 02:00:00,0,2,0.0,NaN,NaN
1179,2019-02-19 03:00:00,0,3,0.0,NaN,NaN
1180,2019-02-19 04:00:00,0,4,0.0,NaN,NaN
...,...,...,...,...,...,...
1339,2019-02-25 19:00:00,0,19,0.0,0.458333,0.25
1340,2019-02-25 20:00:00,0,20,0.0,0.458333,0.25
1341,2019-02-25 21:00:00,0,21,0.0,0.458333,0.25
1342,2019-02-25 22:00:00,0,22,0.0,0.458333,0.25


In [47]:
data_second_test['Passengers_lag_std_24']=data_second_test['Passengers'].rolling(window=24).std()
data_second_test['Passengers_lag_std_48']=data_second_test['Passengers'].rolling(window=48).std()
data_second_test

C:\Users\Ajay\AppData\Local\Temp\ipykernel_4388\2730583247.py:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_second_test['Passengers_lag_std_24']=data_second_test['Passengers'].rolling(window=24).std()

C:\Users\Ajay\AppData\Local\Temp\ipykernel_4388\2730583247.py:2: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_second_test['Passengers_lag_std_48']=data_second_test['Passengers'].rolling(window=48).std()


,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_mean_48,Passengers_lag_std_24,Passengers_lag_std_48
1176,2019-02-19 00:00:00,0,0,0.0,NaN,NaN,NaN,NaN
1177,2019-02-19 01:00:00,0,1,0.0,NaN,NaN,NaN,NaN
1178,2019-02-19 02:00:00,0,2,0.0,NaN,NaN,NaN,NaN
1179,2019-02-19 03:00:00,0,3,0.0,NaN,NaN,NaN,NaN
1180,2019-02-19 04:00:00,0,4,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1339,2019-02-25 19:00:00,0,19,0.0,0.458333,0.25,0.832971,0.635811
1340,2019-02-25 20:00:00,0,20,0.0,0.458333,0.25,0.832971,0.635811
1341,2019-02-25 21:00:00,0,21,0.0,0.458333,0.25,0.832971,0.635811
1342,2019-02-25 22:00:00,0,22,0.0,0.458333,0.25,0.832971,0.635811


In [48]:
data_second_test.fillna(value=0,inplace=True)

C:\Users\Ajay\AppData\Local\Temp\ipykernel_4388\1383842923.py:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_second_test.fillna(value=0,inplace=True)


# Training and test

In [80]:
from pmdarima import auto_arima

In [81]:
data_upsampled[data_upsampled['Passengers']>1]

,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_mean_48,Passengers_lag_std_24,Passengers_lag_std_48
680,2019-01-29 08:00:00,0,8,4.0,0.0,0.0,0.0,0.000000e+00
680,2019-01-29 08:00:00,0,8,4.0,0.0,0.0,0.0,0.000000e+00
680,2019-01-29 08:00:00,0,8,4.0,0.0,0.0,0.0,0.000000e+00
680,2019-01-29 08:00:00,0,8,4.0,0.0,0.0,0.0,0.000000e+00
680,2019-01-29 08:00:00,0,8,4.0,0.0,0.0,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...
1168,2019-02-18 16:00:00,0,16,2.0,2.0,2.0,0.0,1.003139e-07
1168,2019-02-18 16:00:00,0,16,2.0,2.0,2.0,0.0,1.003139e-07
1168,2019-02-18 16:00:00,0,16,2.0,2.0,2.0,0.0,1.003139e-07
1168,2019-02-18 16:00:00,0,16,2.0,2.0,2.0,0.0,1.003139e-07


In [52]:
data_second_month['Passengers_']=data_second_month['Passengers'] - data_second_month['Passengers'].shift(1)

NameError: name 'data_second_month' is not defined

In [ ]:
data_second_month.fillna(value=0,inplace=True)

In [84]:
data_second_month

,date,EZone,hour,Passengers,Passengers_lag_mean_24,Passengers_lag_std,Passengers_
1344,2019-02-26 00:00:00,0,0,0.0,0.458333,0.832971,0.0
1345,2019-02-26 01:00:00,0,1,0.0,0.458333,0.832971,0.0
1346,2019-02-26 02:00:00,0,2,0.0,0.458333,0.832971,0.0
1347,2019-02-26 03:00:00,0,3,0.0,0.458333,0.832971,0.0
1348,2019-02-26 04:00:00,0,4,0.0,0.458333,0.832971,0.0
...,...,...,...,...,...,...,...
1843,2019-03-18 19:00:00,0,19,0.0,0.250000,0.442326,0.0
1844,2019-03-18 20:00:00,0,20,0.0,0.250000,0.442326,0.0
1845,2019-03-18 21:00:00,0,21,0.0,0.250000,0.442326,0.0
1846,2019-03-18 22:00:00,0,22,0.0,0.250000,0.442326,0.0


In [91]:
from statsmodels.tsa.stattools import adfuller
result=adfuller(data_upsampled['Passengers'])

In [92]:
result[1]

1.6184481516961185e-06

**So we can ignore the null hypothesis(data is not stationary) and consider the data as stationary** 

In [123]:
data_second_month.columns

Index(['date', 'EZone', 'hour', 'Passengers', 'Passengers_lag_mean_24',
       'Passengers_lag_mean_48', 'Passengers_lag_std_24',
       'Passengers_lag_std_48'],
      dtype='object')

In [144]:
model=auto_arima(y=data_upsampled['Passengers'],X=data_upsampled[['hour']],trace=True,test='adf',stepwise=True,stationary=True,scoring='mse',seasonal=False,start_p=1, start_q=1)

Performing stepwise search to minimize aic

 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=1683.629, Time=0.33 sec

 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=3621.903, Time=0.27 sec

 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=1694.443, Time=0.25 sec

 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=2778.568, Time=0.24 sec

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=3812.742, Time=0.19 sec

 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=1684.788, Time=0.59 sec

 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=1684.935, Time=1.26 sec

 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=2358.177, Time=1.03 sec

 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=1684.474, Time=0.49 sec

 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=1686.800, Time=3.01 sec

 ARIMA(1,0,1)(0,0,0)[0]             : AIC=1694.814, Time=0.38 sec



Best model:  ARIMA(1,0,1)(0,0,0)[0] intercept

Total fit time: 8.063 seconds


In [139]:
model=auto_arima(y=data_upsampled['Passengers'],trace=True,test='adf',stepwise=True,seasonal=False,scoring='mse',start_p=1, start_q=1)

Performing stepwise search to minimize aic

 ARIMA(1,0,1)(0,0,0)[0]             : AIC=1692.900, Time=0.20 sec

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=4283.370, Time=0.02 sec

 ARIMA(1,0,0)(0,0,0)[0]             : AIC=1708.324, Time=0.06 sec

 ARIMA(0,0,1)(0,0,0)[0]             : AIC=3304.385, Time=0.06 sec

 ARIMA(2,0,1)(0,0,0)[0]             : AIC=1692.776, Time=0.19 sec

 ARIMA(2,0,0)(0,0,0)[0]             : AIC=1694.524, Time=0.09 sec

 ARIMA(3,0,1)(0,0,0)[0]             : AIC=1694.776, Time=0.87 sec

 ARIMA(2,0,2)(0,0,0)[0]             : AIC=1694.776, Time=0.28 sec

 ARIMA(1,0,2)(0,0,0)[0]             : AIC=1693.249, Time=0.13 sec

 ARIMA(3,0,0)(0,0,0)[0]             : AIC=1694.318, Time=0.11 sec

 ARIMA(3,0,2)(0,0,0)[0]             : AIC=1694.236, Time=0.44 sec

 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=1682.789, Time=0.34 sec

 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=1681.631, Time=0.18 sec

 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=2779.082, Time=0.13 sec

 ARIMA(1,0,0)(0,0,

In [145]:
prd=model.predict(X=data_second_test[['hour']],n_periods=168)

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

  data=self.data,


In [140]:
prd=model.predict(n_periods=168)

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

  data=self.data,


In [146]:
prd

988     0.095540
989     0.169503
990     0.239342
991     0.305285
992     0.367550
          ...   
1151    1.425307
1152    1.425104
1153    1.424900
1154    1.424696
1155    1.424492
Length: 168, dtype: float64

In [138]:
np.max(prd.to_numpy())

0.9968042962133048

In [100]:
prd=np.abs(np.round(prd.to_numpy()))

In [101]:
prd

array([0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [147]:
from sklearn.metrics import mean_squared_error
mean_squared_error(data_second_test['Passengers'],prd)

1.5627813355554925

In [122]:
df_busstop0['Passengers'][1848:2016]

1848    0.0
1849    0.0
1850    0.0
1851    0.0
1852    0.0
       ... 
2011    0.0
2012    0.0
2013    0.0
2014    0.0
2015    0.0
Name: Passengers, Length: 168, dtype: float64

In [148]:
np.min(prd.to_numpy())

0.09481297330769135

In [75]:
prd=np.abs(np.round(prd.to_numpy()))
prd

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [76]:
np.max(prd)

1.0

In [58]:
estimator

ARIMA(order=(2, 0, 2), scoring_args={}, suppress_warnings=True)

In [ ]:
estimator

In [73]:
predictions=[]
for train_ind,test_ind in tscv.split(reg_trav_bus_stop0, y=reg_trav_bus_stop0.Passengers, dates = reg_trav_bus_stop0.date):
  estimator=auto_arima(y=df_busstop0.Passengers.iloc[train_ind],X=df_busstop0.drop(columns=['Passengers','EZone','date']).iloc[train_ind],seasonal=True,trace=True,test='adf',stepwise=True,max_order=4)
  estimator.fit(y=df_busstop0.Passengers.iloc[train_ind],X=df_busstop0[["hour", "Passengers_lag_mean","Passengers_lag_std"]].iloc[train_ind])
  prediction_df = df_busstop0[["date", "hour"]].iloc[test_ind].copy()
  prediction_df["Passengers"] = estimator.predict(X=df_busstop0[["hour", "Passengers_lag_mean","Passengers_lag_std"]].iloc[test_ind],n_periods=168)
  predictions.append(prediction_df)

Performing stepwise search to minimize aic

 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=934.338, Time=0.99 sec

 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=943.969, Time=0.19 sec

 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=943.497, Time=0.29 sec

 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=943.625, Time=0.36 sec

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=951.353, Time=0.09 sec

 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=922.808, Time=0.78 sec

 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=945.278, Time=0.78 sec

 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=945.141, Time=0.46 sec

 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=929.786, Time=0.93 sec

 ARIMA(0,0,3)(0,0,0)[0] intercept   : AIC=944.128, Time=0.72 sec

 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=947.091, Time=0.46 sec

 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=935.977, Time=0.88 sec

 ARIMA(1,0,2)(0,0,0)[0]             : AIC=931.376, Time=0.69 sec



Best model:  ARIMA(1,0,2)(0,0,0)[0] intercept

Total fit time: 7.637 seconds

Performing stepwis

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

  data=self.data,


 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=629.744, Time=0.57 sec

 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=637.405, Time=0.10 sec

 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=638.750, Time=0.24 sec

 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=638.879, Time=0.23 sec

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=641.795, Time=0.14 sec

 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=629.836, Time=0.57 sec

 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=642.397, Time=0.57 sec

 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=596.938, Time=0.68 sec

 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=610.353, Time=0.57 sec

 ARIMA(4,0,2)(0,0,0)[0] intercept   : AIC=605.443, Time=1.19 sec

 ARIMA(3,0,3)(0,0,0)[0] intercept   : AIC=599.852, Time=0.97 sec

 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=606.636, Time=0.94 sec

 ARIMA(4,0,1)(0,0,0)[0] intercept   : AIC=622.982, Time=0.95 sec

 ARIMA(4,0,3)(0,0,0)[0] intercept   : AIC=590.510, Time=1.20 sec

 ARIMA(5,0,3)(0,0,0)[0] intercept   : AIC=591.024, Time=1.44 sec

 ARIMA(4,0

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

  data=self.data,


Performing stepwise search to minimize aic

 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=821.550, Time=0.55 sec

 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=814.120, Time=0.14 sec

 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=816.006, Time=0.29 sec

 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=816.011, Time=0.30 sec

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=821.689, Time=0.09 sec

 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=797.670, Time=0.38 sec

 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=798.184, Time=0.45 sec

 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=819.362, Time=0.46 sec

 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=817.765, Time=0.41 sec

 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=817.700, Time=0.36 sec

 ARIMA(1,0,1)(0,0,0)[0]             : AIC=inf, Time=0.10 sec



Best model:  ARIMA(1,0,1)(0,0,0)[0] intercept

Total fit time: 3.546 seconds


...

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

  data=self.data,


Performing stepwise search to minimize aic

 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-7.219, Time=1.00 sec

 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-4.322, Time=0.25 sec

 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-3.541, Time=0.12 sec

 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-3.853, Time=0.28 sec

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-2.835, Time=0.11 sec

 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-3.342, Time=0.74 sec

 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-4.144, Time=0.37 sec

 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=-0.322, Time=0.91 sec

 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=-24.393, Time=0.96 sec

 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-12.687, Time=0.88 sec

 ARIMA(3,0,3)(0,0,0)[0] intercept   : AIC=-29.416, Time=1.03 sec

 ARIMA(4,0,3)(0,0,0)[0] intercept   : AIC=-18.392, Time=1.11 sec

 ARIMA(3,0,4)(0,0,0)[0] intercept   : AIC=-26.381, Time=1.26 sec

 ARIMA(2,0,4)(0,0,0)[0] intercept   : AIC=-32.263, Time=1.19 sec

 ARIMA(1,0,4)(0,0,0)[0] intercept   : AI

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

  data=self.data,


Performing stepwise search to minimize aic

 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=-174.451, Time=0.55 sec

 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-167.282, Time=0.06 sec

 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-165.395, Time=0.12 sec

 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-165.410, Time=0.11 sec

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-169.282, Time=0.06 sec

 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-162.182, Time=0.38 sec

 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-161.880, Time=0.31 sec

 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=-173.029, Time=0.41 sec

 ARIMA(2,0,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.28 sec

 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-173.063, Time=0.26 sec

 ARIMA(1,0,3)(0,0,0)[0] intercept   : AIC=-174.258, Time=0.24 sec

 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=-161.120, Time=0.42 sec

 ARIMA(3,0,3)(0,0,0)[0] intercept   : AIC=-160.291, Time=0.51 sec

 ARIMA(2,0,2)(0,0,0)[0]             : AIC=-171.290, Time=0.36 sec



Best model:  ARIMA(2,

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:843: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.

  data=self.data,


In [61]:
predictions2

,date,hour,Passengers
504,2019-01-22 00:00:00,0,0.273409
505,2019-01-22 01:00:00,1,0.259173
506,2019-01-22 02:00:00,2,0.224758
507,2019-01-22 03:00:00,3,0.193704
508,2019-01-22 04:00:00,4,0.165314
...,...,...,...
8395,2019-12-16 19:00:00,19,NaN
8396,2019-12-16 20:00:00,20,NaN
8397,2019-12-16 21:00:00,21,NaN
8398,2019-12-16 22:00:00,22,NaN


In [62]:
predictions2.isna().sum()

date             0
hour             0
Passengers    2016
dtype: int64

In [ ]:
predictions2.describe()

In [ ]:
for train_ind,test_ind in tscv.split(reg_trav_bus_stop0, y=reg_trav_bus_stop0.Passengers, dates = reg_trav_bus_stop0.date):
  print(train_ind)
  print(test_ind)

In [ ]:
#Encoding EZone in the test data set
regular_travel_test['EZone']=labelencoder.transform(regular_travel_test['EZone'])
regular_travel_test.head()

In [ ]:
df_test=regular_travel_test[regular_travel_test['EZone']==0]

In [ ]:
df_test.reset_index(inplace=True,drop=True)
df_test

In [ ]:
df_test=df_test.groupby(by='date').sum()

In [ ]:
df_test

In [ ]:
data_station1=pd.concat(df,df_test)
data_station1